## Image enhancement with T2T on Kubeflow

### Setup

In [1]:
from tensor2tensor.data_generators import allen_brain
from tensor2tensor.data_generators import allen_brain_utils
from tensorflow.contrib.eager.python import tfe
import tensorflow as tf

import os
import shutil
from tk import util
from tk import jobs
from tk import experiment

import tensorflow as tf

import os
import json

from tk import experiment
from tk import util
import shutil
import json
import logging

from tk.util import hack_dict_to_cli_args

tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys

logging.getLogger().setLevel(logging.INFO)

Instructions for updating:
Use the retry module or similar alternatives.


### TensorBoard

Start a TensorBoard deployment and service pointing to a log_dir parent to all log_dirs we will write for subsequent experiments, allowing us to compare logs from all experiments via a single TensorBoard instance.

In [4]:
from tk import kube_tensorboard

tb = kube_tensorboard.TensorBoard(log_dir="/mnt/nfs-east1-d/testing/foo",
                                  namespace="kubeflow")

tb.create()

# On your local:
# kubectl port-forward --namespace=${KF_NAMESPACE} $PODNAME 8002:80

#kubectl port-forward --namespace=kubeflow tb-j0614-1737-d47b-depl-6ffd568587-4nh6w 8002:80

[2018-06-14 17:45:01,386] Creating object with config: {'kind': 'Service', 'spec': {'ports': [{'targetPort': 80, 'name': 'http', 'port': 80}], 'selector': {'app': 'tensorboard', 'tb-job': 'tb-j0614-1745-7f0d-depl'}}, 'apiVersion': 'v1', 'metadata': {'namespace': 'kubeflow', 'name': 'tb-j0614-1745-7f0d-svc'}}
[2018-06-14 17:45:01,502] Creating object with config: {'kind': 'Deployment', 'spec': {'template': {'spec': {'containers': [{'image': 'tensorflow/tensorflow:1.7.0', 'command': ['/usr/local/bin/tensorboard', '--logdir=/mnt/nfs-east1-d/testing/foo', '--port=80'], 'ports': [{'containerPort': 80}], 'name': 'tensorboard'}]}, 'metadata': {'labels': {'app': 'tensorboard', 'tb-job': 'tb-j0614-1745-7f0d-depl'}, 'namespace': 'kubeflow', 'name': 'tb-j0614-1745-7f0d-depl'}}, 'replicas': 1}, 'apiVersion': 'apps/v1beta1', 'metadata': {'namespace': 'kubeflow', 'name': 'tb-j0614-1745-7f0d-depl'}}


{'api_version': 'v1',
 'kind': 'Service',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 6, 14, 17, 45, 1, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': None,
              'name': 'tb-j0614-1745-7f0d-svc',
              'namespace': 'kubeflow',
              'owner_references': None,
              'resource_version': '1487044',
              'self_link': '/api/v1/namespaces/kubeflow/services/tb-j0614-1745-7f0d-svc',
              'uid': 'a9b0e541-6ffa-11e8-a740-42010a8e0045'},
 'spec': {'cluster_ip': '10.47.254.174',
          'external_i_ps': None,
          'external_name': None,
          'external_traffic_policy': None,
          'health_check_node_port': None,
        

[None, None]

### Training

In [69]:
def gen_local_smoke_args(base_name):

    app_root = "/mnt/nfs-east1-d/work/tk"

    output_dir = os.path.join(app_root, "output")

    job_name = util.generate_job_name(base_name)

    train_args = {
        "problem": "img2img_allen_brain",
        "model": "img2img_transformer",
        "hparams_set": "img2img_transformer2d_tiny",
        "data_dir": "/mnt/nfs-east1-d/data",
        "output_dir": output_dir,
        "train_steps": 100,
        "schedule": "train",
        #"generate_data": True
        "hparams": "batch_size=10",
        "profile": False,
        #"cloud_mlengine": True,
        "log_device_placement": True,
        "worker_gpu": 1,
        "save_checkpoint_secs": 60,
        "dbgprofile": True
    }

    args = {
        "job_name": job_name,
        "volume_claim_id": "nfs-east1-d",
        "app_root": app_root,
        "gcp_project": "foo",
        "namespace": "kubeflow",
        "image": "gcr.io/kubeflow-rl/base:0.0.9",
        "smoke": True,
        "batch": False,
        "train_args": train_args,
        "cpu": 7,
        "memory": "40Gi",
        "num_gpu": 1,
        "num_local_ssd": 1,
        "ssd_mount_path": "/mnt/disks/ssd0",
        "no_wait": True,
        "num_worker_replicas": 0,
        "num_ps_replicas": 0,
        "selector_labels": {
          "cloud.google.com/gke-nodepool": "train-gpu-preemptible-1x-hm",
          "cloud.google.com/gke-accelerator": "nvidia-tesla-k80"
        }
    }

    return args


def _stage(local_app_root, remote_app_root):
    if not os.path.exists(local_app_root):
        raise ValueError("Can't stage from a non-existent source, "
                         "saw %s" % local_app_root)
    shutil.copytree(local_app_root, remote_app_root)


def gen_remote_smoke_args(base_name):

    args = gen_local_smoke_args(base_name)
    local_app_root = args["app_root"]

    testing_storage_base = "/mnt/nfs-east1-d/testing"
    remote_app_root = "%s/%s" % (testing_storage_base,
                                 args["job_name"])

    args["train_args"]["output_dir"] = os.path.join(remote_app_root,
                                                    "output")

    with open(os.path.join(local_app_root, "job.sh"), "w") as f:
      f.write("pip install -e %s\n" % remote_app_root)
      cmd = ["python", "-m", "tk.experiment"]
      cmd.extend(hack_dict_to_cli_args(args["train_args"]))
      f.write(" ".join(cmd) + "\n")
      logging.info(local_app_root)

    _stage(local_app_root, remote_app_root)
    args["app_root"] = remote_app_root
    args["batch"] = True

    return args


In [60]:
args = gen_remote_smoke_args("profiles")

[2018-06-16 00:16:00,318] /mnt/nfs-east1-d/work/tk


In [61]:
args

{'app_root': '/mnt/nfs-east1-d/testing/profiles-j0616-0016-0620',
 'batch': True,
 'cpu': 6,
 'gcp_project': 'foo',
 'image': 'gcr.io/kubeflow-rl/base:0.0.9',
 'job_name': 'profiles-j0616-0016-0620',
 'namespace': 'kubeflow',
 'no_wait': True,
 'num_gpu': 1,
 'num_local_ssd': 1,
 'num_ps_replicas': 0,
 'num_worker_replicas': 0,
 'selector_labels': {'cloud.google.com/gke-accelerator': 'nvidia-tesla-k80',
  'cloud.google.com/gke-nodepool': 'train-gpu-preemptible-1x-hm'},
 'smoke': True,
 'ssd_mount_path': '/mnt/disks/ssd0',
 'train_args': {'data_dir': '/mnt/nfs-east1-d/data',
  'dbgprofile': True,
  'hparams': 'batch_size=10',
  'hparams_set': 'img2img_transformer2d_tiny',
  'log_device_placement': True,
  'model': 'img2img_transformer',
  'output_dir': '/mnt/nfs-east1-d/testing/profiles-j0616-0016-0620/output',
  'problem': 'img2img_allen_brain',
  'profile': False,
  'save_checkpoint_secs': 60,
  'schedule': 'train',
  'train_steps': 100,
  'worker_gpu': 1},
 'volume_claim_id': 'nfs-ea

In [62]:
job = experiment.T2TExperiment(**args)
job.run()

[2018-06-16 00:17:07,531] smoke: True


In [70]:
for i in range(0, 4):
    args = gen_remote_smoke_args("multinode")
    job = experiment.T2TExperiment(**args)
    job.run()

[2018-06-18 16:55:28,676] /mnt/nfs-east1-d/work/tk
[2018-06-18 16:56:50,821] smoke: True
[2018-06-18 16:56:50,946] /mnt/nfs-east1-d/work/tk
[2018-06-18 16:58:12,952] smoke: True
[2018-06-18 16:58:13,074] /mnt/nfs-east1-d/work/tk
[2018-06-18 16:59:35,686] smoke: True
[2018-06-18 16:59:35,814] /mnt/nfs-east1-d/work/tk
[2018-06-18 17:00:59,650] smoke: True
